# Лабораторна робота 5. Варіант 1.
# ІП-13 Ал Хадам Мурат
# Моделювання тем
## Мета роботи: Ознайомитись з вирішенням задач пошуку ключових слів та моделювання тем.
1. Застосувати приховане семантичне індексування бібліотеки Gensim для моделювання тем. Вивести документи, що зробили найбільший вклад в теми. Обрати або створити три нових документи (які модель ще не бачила) та визначити їх теми.
2. Використати текст austen-persuasion.txt з корпусу gutenberg бібліотеки nltk та вивести ключові біграми.

In [1]:
import pandas as pd
data = pd.read_csv('news.csv')
data

,text,label
0,Here are Thursday's biggest analyst calls: App...,0
1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,"Analysts react to Tesla's latest earnings, bre...",0
4,Netflix and its peers are set for a ‘return to...,0
...,...,...
16985,KfW credit line for Uniper could be raised to ...,3
16986,KfW credit line for Uniper could be raised to ...,3
16987,Russian https://t.co/R0iPhyo5p7 sells 1 bln r...,3
16988,Global ESG bond issuance posts H1 dip as supra...,3


In [2]:
texts = data['text'].tolist()

In [3]:
from gensim.parsing.preprocessing import preprocess_string
from gensim import corpora

# Попередня обробка текстів
processed_texts = [preprocess_string(text) for text in texts]

# Створення словника та корпусу
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

Створити модель з модуля gensim

In [4]:
from gensim.models import LsiModel

# Побудова LSI моделі
lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=10)

Вивести документи, що зробили найбільший вклад в теми.

In [5]:
topics = lsi_model.print_topics(num_topics=10, num_words=5)
for i, topic in topics:
    print(f"Topic #{i}: {topic}")

# Визначення внеску кожного документа в теми
corpus_lsi = lsi_model[corpus]

# Визначення топ документів для кожної теми
from collections import defaultdict

topic_contributions = defaultdict(list)
for doc_id, doc in enumerate(corpus_lsi):
    for topic_id, contribution in doc:
        topic_contributions[topic_id].append((contribution, doc_id))

for topic_id, contributions in topic_contributions.items():
    top_docs = sorted(contributions, reverse=True)[:3]
    print(f"\nTop documents for topic #{topic_id}:")
    for contribution, doc_id in top_docs:
        print(f"\nDocument #{doc_id} with contribution {contribution}: {texts[doc_id]}")

Topic #0: 0.966*"http" + 0.060*"announc" + 0.058*"market" + 0.056*"new" + 0.055*"stock"
Topic #1: -0.508*"stock" + -0.469*"market" + -0.295*"trade" + -0.264*"economi" + -0.250*"invest"
Topic #2: 0.377*"quarter" + 0.354*"second" + 0.319*"result" + 0.291*"earn" + 0.251*"announc"
Topic #3: -0.351*"rate" + -0.341*"inflat" + -0.304*"quarter" + -0.283*"second" + -0.260*"year"
Topic #4: 0.629*"new" + 0.427*"est" + 0.290*"prev" + 0.263*"jun" + -0.257*"market"
Topic #5: -0.730*"market" + 0.438*"stock" + -0.236*"est" + -0.160*"jun" + -0.159*"prev"
Topic #6: -0.680*"new" + 0.392*"est" + 0.229*"prev" + 0.216*"jun" + -0.182*"market"
Topic #7: -0.456*"price" + -0.419*"stock" + 0.396*"bank" + 0.246*"rate" + 0.178*"invest"
Topic #8: -0.473*"stock" + 0.294*"price" + 0.289*"trade" + 0.258*"economi" + -0.256*"market"
Topic #9: -0.410*"beat" + 0.378*"announc" + -0.338*"revenu" + -0.300*"ep" + -0.294*"earn"

Top documents for topic #0:

Document #927 with contribution 3.8801262379041486: NovaBay Pharmaceut

In [6]:
new_texts = [
    "Analysts say Apple will continue to grow in the next quarter.",
    "Tesla's new model has impressed the market with its advanced features.",
    "Amazon's stock prices are predicted to rise due to increased sales."
]

new_processed_texts = [preprocess_string(text) for text in new_texts]
new_corpus = [dictionary.doc2bow(text) for text in new_processed_texts]
new_corpus_lsi = lsi_model[new_corpus]

for i, doc in enumerate(new_corpus_lsi):
    print(f"New document #{i + 1}: {new_texts[i]}")
    for topic_id, contribution in doc:
        print(f" - Topic #{topic_id + 1} with contribution {contribution}")

New document #1: Analysts say Apple will continue to grow in the next quarter.
 - Topic #1 with contribution 0.0701707757892538
 - Topic #2 with contribution 0.02605186813960929
 - Topic #3 with contribution 0.33907283961691104
 - Topic #4 with contribution -0.3150724896901351
 - Topic #5 with contribution 0.009183449897806381
 - Topic #6 with contribution -0.08875207098864706
 - Topic #7 with contribution -0.11517570500442723
 - Topic #8 with contribution -0.13454558586140025
 - Topic #9 with contribution 0.03997909510178691
 - Topic #10 with contribution 0.008109876492719797
New document #2: Tesla's new model has impressed the market with its advanced features.
 - Topic #1 with contribution 0.12970955198419243
 - Topic #2 with contribution -0.4276164854458416
 - Topic #3 with contribution -0.23161052124978343
 - Topic #4 with contribution 0.16704065259199743
 - Topic #5 with contribution 0.38628609795852714
 - Topic #6 with contribution -0.7702722717163895
 - Topic #7 with contributi

2. Використати текст austen-persuasion.txt з корпусу gutenberg бібліотеки nltk та вивести ключові біграми.

In [7]:
import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
import string

# Завантажимо текст
nltk.download('gutenberg')
nltk.download('punkt')
persuasion_text = gutenberg.raw('austen-persuasion.txt')
stop_words = set(stopwords.words('english'))

# Токенізація
tokens = nltk.word_tokenize(persuasion_text)
cleaned_tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation and token not in ['``', "''"]]

# Знаходження біграм
bigram_finder = BigramCollocationFinder.from_words(cleaned_tokens)
bigrams = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)

print("Key Bigrams:")
for bigram in bigrams:
    print(bigram)


[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\murat\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\murat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Key Bigrams:
('Captain', 'Wentworth')
('Lady', 'Russell')
('Sir', 'Walter')
('Mr', 'Elliot')
('Mrs', 'Clay')
('Mrs', 'Smith')
('Captain', 'Benwick')
('Mrs', 'Musgrove')
('Camden', 'Place')
('great', 'deal')
